In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

In [3]:
geolocator = Nominatim(user_agent="MyApp")

In [4]:
col = pd.read_csv('ColfSelecClean.csv')

In [10]:
col['Ciudad Destino'].value_counts()

londres             1594
new york             728
barcelona            677
paris                616
london               554
                    ... 
dekalb                 1
chestnut hill ma       1
bremerhaven            1
mons                   1
shepherdstown          1
Name: Ciudad Destino, Length: 939, dtype: int64

In [12]:
col[col['lat'].isna()]['Ciudad Destino'].value_counts()

goterborg                     7
glendele                      3
kensington &amp; chelsea      3
freiburg im breisgrau         2
reykiavik                     2
meford                        1
leicestel                     1
wivenhoe park, colc           1
guilfport                     1
mont sain again               1
villejiuf                     1
lyngbi                        1
jouy en josasjouy-en-josas    1
bron cedex                    1
Name: Ciudad Destino, dtype: int64

In [32]:
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('amp;','')

In [44]:
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('leicestel','leicester')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('goterborg','Gotemburgo')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('glendele','Glendale')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('reykiavik','Reikiavik')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('meford','Medford')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('freiburg im breisgrau','Friburgo de Brisgovia')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('wivenhoe park, colc','Coventry')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('guilfport','Gulfport')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('mont sain again','Mont-Saint-Aignan')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('villejiuf','Villejuif')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('lyngbi','Lyngby')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('jouy en josasjouy-en-josas','jouy-en-josas')
col['Ciudad Destino'] = col['Ciudad Destino'].str.replace('bron cedex','Lyon')

In [25]:
geolocator.geocode(['Gotemburgo','Suecia'])

Location(Göteborg, Göteborgs Stad, Västra Götalands län, 411 10, Sverige, (57.7072326, 11.9670171, 0.0))

In [26]:
geolocator.geocode(['Reikiavik','Island'])

Location(Reykjavíkurborg, Höfuðborgarsvæðið, Ísland, (64.145981, -21.9422367, 0.0))

In [31]:
geolocator.geocode(['Glendale','Arizona'])

Location(Glendale, Maricopa County, Arizona, United States, (33.5386858, -112.185994, 0.0))

In [34]:
geolocator.geocode('kensington & chelsea')

Location(Chelsea, Royal Borough of Kensington and Chelsea, London, Greater London, England, SW3 5UA, United Kingdom, (51.4875167, -0.1687007, 0.0))

In [43]:
col[(col['lat'].isna()) & (col['Ciudad Destino'] == 'bron cedex')]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon
9203,2015,Aura Liceth Hoyos Ariza,Femenino,Quindío,Universidad del Quindío,Armenia,Licenciado en Idiomas,Lyon 2 - Université Lumière,Francia,bron cedex,Maestría,Sciences du Langage: Enseignement et Diffusion...,Educación,seleccionado,NaN,NaN


In [45]:
unique_cities = col[col['lat'].isna()]['Ciudad Destino'].unique()

In [46]:
cites_location = {}
for i in unique_cities:
    location = geolocator.geocode(i)
    if location:
        cites_location[i] = (location.latitude,location.longitude)
    else:
        cites_location[i] = None

In [56]:
def cities_dict(row):
    x = row['Ciudad Destino']
    try:
        if cites_location[x]:
            return cites_location[x][0],cites_location[x][1]
        else:
            return row['lat'],row['lon']
    except:
        return row['lat'],row['lon']

In [57]:
col['lat'],col['lon'] = zip(*col.apply(lambda x: cities_dict(x),axis=1))

In [58]:
col[col['lat'].isna()]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon


In [66]:
col['loc'] = col['lat'].astype(str)+ ',' +col['lon'].astype(str)

In [98]:
col['loc'].value_counts()

51.5073219,-0.1276474            2149
40.7127281,-74.0060152           1113
41.3828939,2.1774322              678
48.8588897,2.3200410217200766     616
45.4641943,9.1896346              525
                                 ... 
43.1070321,12.402996209906489       1
-21.9980468,-47.4280861             1
42.6985304,2.8953121                1
60.4517531,22.2670522               1
39.4300996,-77.804161               1
Name: loc, Length: 793, dtype: int64

In [102]:
def get_city(location):
    #location = row['loc']
    loc = geolocator.reverse(location)
    loc = loc.raw['address']
    if 'city' in loc:
        city = loc['city']
    elif 'town' in loc:
        city = loc['town']
    elif 'village' in loc:
        city = loc['village']
    elif 'county' in loc:
        city = loc['county']
    elif 'city_district' in loc:
        city = loc['city_district']
    elif 'state' in loc:
        city = loc['state']
    elif 'country' in loc:
        city = loc['country']
    else:
        print(loc)
        return None
    return city

In [100]:
col['Ciudad Destino2'] = col.apply(lambda x: get_city(x),axis=1)

KeyboardInterrupt: 

In [104]:
unique_cities_loc

array(['40.4416941,-79.9900861', '40.6526006,-73.9497211',
       '48.8588897,2.3200410217200766', '40.4167047,-3.7035825',
       '51.5073219,-0.1276474', '53.9590555,-1.0815361',
       '40.7127281,-74.0060152', '33.7489924,-84.3902644',
       '45.5031824,-73.5698065', '39.2908816,-76.610759',
       '38.8950368,-77.0365427', '35.9131542,-79.05578',
       '25.7741728,-80.19362', '53.7974185,-1.5437941',
       '45.4208777,-75.6901106', '51.8896903,0.8994651',
       '56.1181242,-3.9360012', '-33.8698439,151.2082848',
       '39.1670396,-86.5342881', '52.2055314,0.1186637',
       '30.2711286,-97.7436995', '43.157285,-77.615214',
       '53.4794892,-2.2451148', '53.550341,10.000654',
       '4.099917,-72.9088133', '32.753177,-97.3327459',
       '43.6534817,-79.3839347', '-37.8142176,144.9631608',
       '42.498275,-92.3328743', '51.7520131,-1.2578499',
       '37.7790262,-122.419906', '42.0447388,-87.6930459',
       '51.148555,0.8722566', '53.4071991,-2.99168',
       '43.3271048,

In [105]:
unique_cities_loc = col['loc'].unique()
cities_location = {}
for i in unique_cities_loc:
    cities_location[i] = get_city(i)

In [86]:
unique_cities = col['Ciudad Destino'].unique()

In [107]:
col['Ciudad_Destino'] = col['loc'].apply(lambda x : cities_location[x])

In [97]:
geolocator.geocode('chelsie').address

'Chelsie Court, Fore Meadows, Bowling Green, Wood County, Ohio, 43402, United States'

In [110]:
col[col['Ciudad_Destino'].isna()]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino


In [111]:
col.to_csv('ColfSelecClean.csv',index=False)

In [112]:
col[['Ciudad Destino','Ciudad_Destino']]

,Ciudad Destino,Ciudad_Destino
0,pittsburgh,Pittsburgh
1,brooklyn,City of New York
2,pittsburgh,Pittsburgh
3,pittsburgh,Pittsburgh
4,paris,Paris
...,...,...
19301,lisboa,Lisboa
19302,londres,City of Westminster
19303,hamilton,Hamilton
19304,beaconsfield,Beaconsfield


In [116]:
col[col['Ciudad_Destino'] == 'London']

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino
2721,2009,Jorge Enrique León González,Masculino,Distrito Capital,Universidad Santo Tomás de Aquino,Bogotá,Ingeniero Electrónico,Brunel University,Reino Unido,uxbridge,Maestría,Data Communications Systems,Ingeniería,beneficiario,51.544951,-0.481667,"51.5449509,-0.4816672",London
2750,2009,Nicolas Rico Rojas,Masculino,Distrito Capital,Pontificia Universidad Javeriana,Bogotá,Maestro en Música,University of Westminster,Reino Unido,middlesex,Maestría,Audio Production,Artes,beneficiario,51.546056,-0.253779,"51.546055800000005,-0.2537791389340719",London
2781,2010,Ivo Alejandro Valencia Bernal,Masculino,Tolima,Universidad Nacional De Colombia,Bogotá,Ingeniero Civil,University of New South Wales,Australia,kensington,Maestría,Engineering Science,Ingeniería,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
2942,2010,Diego Alejandro Chacón Fajardo,Masculino,Distrito Capital,Universidad Militar-Nueva Granada,Bogotá,Ingeniero de Telecomunicaciones,University of New South Wales,Australia,kensington,Maestría,Engineering Science,Ingeniería,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
3179,2010,María Juliana Gómez Osorio,Femenino,Santander,Universidad Pontificia Bolivariana,Bucaramanga,Ingeniero Ambiental y Sanitario,Brunel University,Reino Unido,uxbridge,Maestría,Occupational Health and Safety Management MSc,Ciencias de la Salud,beneficiario,51.544951,-0.481667,"51.5449509,-0.4816672",London
3486,2010,Juan Nicolás Quiñones Hoyos,Masculino,Distrito Capital,Universidad De Los Andes,Bogotá,Administrador de Empresas,University of New South Wales,Australia,kensington,Maestría,Environmental Management,Ciencias Agropecuarias y del Medio Ambiente,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
3512,2010,Pedro Alonso Ramírez Cardozo,Masculino,Bolívar,Universidad De Los Andes,Bogotá,Ingeniero Químico,University of New South Wales,Australia,kensington,Maestría,Chemical Process Engineering,Ingeniería,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
3535,2010,Natalia Gaviria Rubio,Femenino,Distrito Capital,Pontificia Universidad Javeriana,Bogotá,Arquitecto,University of New South Wales,Australia,kensington,Maestría,Sustainable Development,Arquitectura y Diseño,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
3611,2010,Luis Gabriel Bernal Pulido,Masculino,Distrito Capital,Universidad El Bosque,Bogotá,Médico Cirujano,University of New South Wales,Australia,kensington,Maestría,Public Health,Ciencias de la Salud,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London
3650,2010,Andrés Alvarez Raigosa,Masculino,Antioquia,Escuela De Ingeniería De Antioquia,Medellín,Ingeniero Industrial,University of New South Wales,Australia,kensington,Maestría,Commerce,Administración y Negocios,beneficiario,51.498995,-0.199123,"51.4989948,-0.1991229",London


In [117]:
col['Ciudad_Destino'] = col['Ciudad_Destino'].str.replace('City of Westminster','London')

In [118]:
col.to_csv('ColfSelecClean.csv',index=False)

In [140]:
col = pd.read_csv('ColfSelecClean.csv')

In [141]:
col['País'].value_counts()

EE.UU                     4456
Reino Unido               4087
Alemania                  2062
Francia                   1499
España                    1479
Australia                 1252
Italia                    1101
Países bajos               921
Canadá                     610
Suecia                     256
Países Bajos               253
Brasil                     224
Bélgica                    219
Suiza                      175
México                      87
Nueva Zelanda               73
Dinamarca                   59
Portugal                    56
Noruega                     52
Finlandia                   48
Argentina                   40
Chile                       37
Austria                     36
China                       33
República de Singapur       28
Costa Rica                  20
República Checa             18
Irlanda                     15
Hungría                     14
Japón                       12
Islandia                    10
Rusia                       10
Singapur

In [142]:
col['Área'].value_counts()

Ingeniería                                                                    4130
Administración y Negocios                                                     2584
Ciencias Sociales                                                             2017
Derecho                                                                       1612
Arquitectura y Diseño                                                         1441
Ciencias Políticas y Relaciones Internacionales                               1393
Artes                                                                         1302
Ciencias Básicas                                                              1193
Ciencias Agropecuarias y del Medio Ambiente                                   1137
Ciencias de la Salud                                                           935
Educación                                                                      834
Economía                                                                       699
Dere

In [143]:
col['Área'] = col['Área'].str.replace('\r','')
col['Área'] = col['Área'].str.replace('\n','')
col['Área'] = col['Área'].str.replace('CIENCIAS AGROPECUARIAS Y DEL MEDIO AMBIENTE','Ciencias Agropecuarias y del Medio Ambiente')
col['Área'] = col['Área'].str.replace('ARQUITECTURA Y DISEÑO','Arquitectura y Diseño')
col['Área'] = col['Área'].str.replace('C.Agropecuarias','Ciencias Agropecuarias y del Medio Ambiente')
col['Área'] = col['Área'].str.replace('(','')
col['Área'].replace('IN  SELECT * FROM  SELECT',' ',regex=True,inplace=True)
col['Área'] = col['Área'].str.replace('Ciencias Políticas y Relacion es Internacionales','Ciencias Políticas y Relaciones Internacionales',regex=True)

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_27668/2440945096.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Área'] = col['Área'].str.replace('C.Agropecuarias','Ciencias Agropecuarias y del Medio Ambiente')
C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_27668/2440945096.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  col['Área'] = col['Área'].str.replace('(','')


In [144]:
col[col['Área'] == 'Ciencias Políticas y RelacionIN  SELECT * FROM  SELECTes Internacionales']

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino
15202,2020,Francisco Javier Escobar Onofre,Masculino,Bogotá,Universidad de Los Andes,Bogotá,Profesional en Gobierno y Asuntos Públicos,London School of Economics and Political Science,Reino Unido,londres,Maestría,International Political Economy,Ciencias Políticas y RelacionIN SELECT * FROM...,beneficiario,51.507322,-0.127647,"51.5073219,-0.1276474",London


In [148]:
col.loc[15202,'Área'] = 'Ciencias Políticas y Relaciones Internacionales'

In [149]:
col['Área'].value_counts()

Ingeniería                                                  4131
Administración y Negocios                                   2585
Ciencias Sociales                                           2017
Derecho                                                     1612
Arquitectura y Diseño                                       1442
Ciencias Políticas y Relaciones Internacionales             1394
Artes                                                       1303
Ciencias Básicas                                            1193
Ciencias Agropecuarias y del Medio Ambiente                 1140
Ciencias de la Salud                                         935
Educación                                                    834
Economía                                                     699
Derecho, Ciencias Políticas y Relaciones Internacionales      21
Name: Área, dtype: int64

In [150]:
col.to_csv('ColfSelecClean.csv',index=False)

In [152]:
col['Tipo'].value_counts()

Maestría                       16060
Doctorado                       2235
Maestría en Administración       849
Especialización                  145
Doctorado en Administración       13
Maestría\r\n                       2
Doctorado\r\n                      1
Maestris                           1
Name: Tipo, dtype: int64

In [153]:
col['Tipo'] = col['Tipo'].str.replace('\r','')
col['Tipo'] = col['Tipo'].str.replace('\n','')
col['Tipo'] = col['Tipo'].str.replace('Maestris','Maestría')

In [154]:
col.to_csv('ColfSelecClean.csv',index=False)

In [102]:
col = pd.read_csv('ColfSelecClean.csv')

In [103]:
col['Univ. Posgrado'].unique().tolist()

['Carnegie Mellon University',
 'Pratt Institute',
 'École des Hautes Études en Sciences Sociales (EHESS)',
 'Instituto de Empresa (IE)',
 'University College London (UCL)',
 'London School of Economics &amp; Political Science',
 'University of York',
 'Columbia University',
 'Georgia Institute of Technology',
 'McGill University',
 'Johns Hopkins University',
 'Georgetown University',
 'University of North Carolina',
 'Florida Internacional University',
 'Imperial College',
 'University of Leeds',
 'Carleton University',
 'University of Essex',
 'University of Stirling',
 'Parsons School of Desing',
 'University of Sydney',
 'Indiana University',
 'Harvard University',
 'University of Texas',
 'Carlos III de Madrid',
 'University of Rochester',
 'University of Manchester',
 'Tecnische Univertität Hamburg-Harburg',
 'The New School University',
 'University of South Carolina',
 'The New School',
 'Texas Christian University',
 'Massachusetts Institute of Technology - MIT',
 'University

In [105]:
col[col['Univ. Posgrado'].str.contains('U\.')]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino
237,2002,Germán Montoya Valenzuela,Masculino,Cundinamarca,Cornell U.,Ithaca,Bachelor Of Science,Tufts U.,EE.UU,boston,Maestría,Law and Diplomacy,"Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,42.355433,-71.060511,"42.3554334,-71.060511",Boston
239,2002,Luz Stella Rodríguez Novoa,Femenino,Cundinamarca,Externado,Bogotá,Prof. Finanzas y Relaciones Internacionales,Columbia U.,EE.UU,new york,Maestría,Economic Policy Management,"Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,40.712728,-74.006015,"40.7127281,-74.0060152",City of New York
251,2002,Isabel Cristina Garcés Palacio,Femenino,Antioquia,Javeriana,Bogotá,Nutricionista Dietista,U. of Alabama at Birmingham,EE.UU,birmingham,Maestría,Maternal and Child Health Enhanced Skill,Ciencias de la Salud,seleccionado,52.479699,-1.902691,"52.4796992,-1.9026911",Birmingham
265,2002,Lope Hugo Barrero Solano,Masculino,Huila,Javeriana,Bogotá,Ing. Industrial,Harvard U.,EE.UU,cambridge,Maestría,Environmental Health - Concentration in Enviro...,Ingeniería,seleccionado,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
268,2002,Claudia Patricia Pabón Pereira,Femenino,Santander,Javeriana,Bogotá,Ing. Industrial,Wageningen U.,Países bajos,wageningen,Maestría,Environmental Sciences,Ciencias Agropecuarias y del Medio Ambiente,seleccionado,51.968582,5.668298,"51.9685817,5.668298061827958",Wageningen
272,2002,Carlos Mauricio Gaviria Cataño,Masculino,Antioquia,Antioquia,Medellín,Ing. Electronico,Monash U.,Australia,melbourne,Maestría,MBA,Administración y Negocios,seleccionado,-37.814218,144.963161,"-37.8142176,144.9631608",Melbourne
279,2002,Jimmy Edgard Alvarez Diaz,Masculino,Cundinamarca,Nacional,Bogotá,Biólogo,U. Of Kent At Canterbury,Reino Unido,canterbury,Maestría,Conservation Biology,Ciencias Agropecuarias y del Medio Ambiente,seleccionado,51.280028,1.080253,"51.2800275,1.0802533",Canterbury
297,2002,Maria Carolina Posada Velandia,Femenino,Atlantico,Andes,Bogotá,Abogado,U. Of Essex,Reino Unido,essex,Maestría,LL.M in International Human Rights Law,"Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,51.770468,0.464670,"51.77046785,0.4646697741230038",Chelmsford
298,2002,Nathalia Rueda Linares,Femenino,Cundinamarca,Andes,Bogotá,Ing. Industrial,Georgetown U.,EE.UU,washington,Maestría,MBA,Administración y Negocios,seleccionado,38.895037,-77.036543,"38.8950368,-77.0365427",Washington
301,2002,Diana Milena Guio Torres,Femenino,Cundinamarca,Nacional,Bogotá,Ing. Químico,U. De Plymouth,Reino Unido,plymouth,Maestría,Holistic Science,Ciencias Agropecuarias y del Medio Ambiente,seleccionado,50.371266,-4.142566,"50.3712659,-4.1425658",Plymouth


In [106]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('&amp;','')
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('Of','of')
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('OF','of')

In [107]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.strip()

In [108]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('Massachusetts Institute of Technology','Massachusetts Institute of Technology - MIT')

In [109]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('- MIT - MIT','- MIT')


In [110]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('\xa0',' ')

In [111]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('- MIT \(MIT\)','- MIT')

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_16668/248367895.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('- MIT \(MIT\)','- MIT')


In [112]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('U\.','University')

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_16668/4148585123.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Univ. Posgrado'] = col['Univ. Posgrado'].str.replace('U\.','University')


In [113]:
col[col['Univ. Posgrado'].str.contains('MIT')]['Univ. Posgrado'].unique()

array(['Massachusetts Institute of Technology - MIT', 'RMIT University',
       'Royal Melbourne Institute of Technology - RMIT',
       'Royal Melbourne Institute of Technology (RMIT) University',
       'Royal Melbourne Institute of Technology (RMIT)',
       'Royal Melbourne Institute of Technology - RMIT University',
       'Royal Melbourne Institute of Technology (RMIT University)'],
      dtype=object)

In [83]:
import universities

uni = universities.API()

In [91]:
uni.lucky(name = "UNIVERSITY OF QUEENSLAND",country='Australia')

universities.models.University(name='University of Queensland', domains=['uq.edu.au'], web_pages=['http://www.uq.edu.au/'], country_code='AU', stateprov=None, country='Australia')

In [95]:
uni.lucky(name = "Massachusetts Institute of Technology - MIT",country='EE.UU')

In [86]:
print(mit)

<generator object API.search at 0x000001C601433E40>


In [87]:
mit.__str__

<method-wrapper '__str__' of generator object at 0x000001C601433E40>

In [88]:
uni.search(country='Canada')

<generator object API.search at 0x000001C601433F90>

In [114]:
col['Univ. Posgrado'].unique().tolist()

['Carnegie Mellon University',
 'Pratt Institute',
 'École des Hautes Études en Sciences Sociales (EHESS)',
 'Instituto de Empresa (IE)',
 'University College London (UCL)',
 'London School of Economics  Political Science',
 'University of York',
 'Columbia University',
 'Georgia Institute of Technology',
 'McGill University',
 'Johns Hopkins University',
 'Georgetown University',
 'University of North Carolina',
 'Florida Internacional University',
 'Imperial College',
 'University of Leeds',
 'Carleton University',
 'University of Essex',
 'University of Stirling',
 'Parsons School of Desing',
 'University of Sydney',
 'Indiana University',
 'Harvard University',
 'University of Texas',
 'Carlos III de Madrid',
 'University of Rochester',
 'University of Manchester',
 'Tecnische Univertität Hamburg-Harburg',
 'The New School University',
 'University of South Carolina',
 'The New School',
 'Texas Christian University',
 'Massachusetts Institute of Technology - MIT',
 'University of T

In [96]:
def look_uni(row):
    ru = uni.lucky(name = row['Univ. Posgrado'],country= row['País'])
    if ru:
        return ru.name
    else:
        return row['Univ. Posgrado']

In [98]:
col['UniPos2'] = col.apply(lambda x: look_uni(x),axis=1)

KeyboardInterrupt: 

In [120]:
col[col['Univ. Posgrado'].str.contains('Harvard')]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino
24,2000,Sandra María Sarria Medina,Femenino,Distrito Capital,Pontificia Universidad Javeriana,Bogotá,Abogada,Harvard University,EE.UU,cambridge,Maestría,Public Administration,Ciencias Políticas y Relaciones Internacionales,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
75,2000,María Carolina Nieto Ángel,Femenino,Distrito Capital,Universidad Sergio Arboleda,Bogotá,Abogada,Harvard University,EE.UU,cambridge,Maestría,International Education Policy,Educación,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
78,2000,Catalina Ortíz Lalinde,Femenino,Valle,Universidad De Los Andes,Bogotá,Abogada,Harvard University,EE.UU,cambridge,Maestría,Public Administration,Ciencias Políticas y Relaciones Internacionales,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
160,2001,Carlos Alberto Casas Herrera,Masculino,Distrito Capital,Pontificia Universidad Javeriana,Bogotá,Abogado,Harvard University,EE.UU,cambridge,Maestría,Education in School Leadership,Educación,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
165,2001,Maria Claudia Orjuela,Femenino,Distrito Capital,Pontificia Universidad Javeriana,Bogotá,Comunicadora Social,Harvard University,EE.UU,cambridge,Maestría,International Education Policy,Educación,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18748,2022,Daniela Campos Rojas,Femenino,Bogotá,Universidad de Los Andes,Bogotá,Politologa,Harvard University,EE.UU,cambridge,Maestría,Public Policy,Ciencias Políticas y Relaciones Internacionales,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
18865,2022,Nicolás Caballero Hernández,Femenino,Bogotá,Pontificia Universidad Javeriana (PUJ),Bogotá,Historiador,Harvard University,EE.UU,cambridge,Maestría,LLM,Derecho,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
19031,2022,Ricardo Mauricio Díaz Alarcón,Masculino,Bogotá,Universidad de Los Andes,Bogotá,Literato,Harvard University,EE.UU,cambridge,Maestría,LLM,Derecho,beneficiario,52.205531,0.118664,"52.2055314,0.1186637",Cambridge
19230,2022,Catalina Meléndez Contreras,Femenino,Bogotá,Universidad de Los Andes,Bogotá,Médico,Harvard University,EE.UU,boston,Maestría,Public Health: Global Health,Ciencias de la Salud,beneficiario,42.355433,-71.060511,"42.3554334,-71.060511",Boston


In [123]:
col['País'].value_counts()

EE.UU                     4456
Reino Unido               4087
Alemania                  2062
Francia                   1499
España                    1479
Australia                 1252
Países Bajos              1174
Italia                    1101
Canadá                     610
Suecia                     256
Brasil                     224
Bélgica                    220
Suiza                      175
México                      87
Nueva Zelanda               73
Dinamarca                   59
Portugal                    56
Noruega                     52
Finlandia                   48
Argentina                   40
Austria                     37
Chile                       37
China                       33
República de Singapur       28
República Checa             21
Costa Rica                  20
Irlanda                     16
Hungría                     14
Japón                       12
Islandia                    10
Rusia                       10
Singapur                     9
Estonia 

In [122]:
col['País'] = col['País'].str.replace('Viena','Austria')
col['País'] = col['País'].str.replace('Belgica','Bélgica')
col['País'] = col['País'].str.replace('Ireland','Irlanda')
col['País'] = col['País'].str.replace('Chequia','República Checa')
col['País'] = col['País'].str.replace('bajos','Bajos')

In [124]:
col.to_csv('ColfSelecClean.csv',index=False)

In [140]:
col['Posgrado'].unique().tolist()

['Industrial Administration',
 'Industrial design',
 'MBA Industrial Administration',
 'Information Systems Management',
 'Sociología',
 'MBA Internacional',
 'Development and Planning',
 'Decision Sciences',
 'Sociology',
 'Arts',
 'Civil Engineering',
 'MBA',
 'International Relations and International Economics',
 'LLM in Securities and Financial Regulation',
 'Epidemiology',
 'Administración Hotelera',
 'Soil Mechanics',
 'Ecology and Evolution ',
 'International Affairs',
 'Art History and Theory',
 'Edición',
 'Lighting Design',
 'Aeronautical Engineering',
 'Public Administration',
 'Politica territorial y urbanistica',
 'Neuroscience',
 'Tecnología de la Información',
 'Maestría en Piano',
 'Public Policy',
 'Juridical Science',
 'International Business',
 'Opera Conducting',
 'Piano',
 'Minimium cost housing for developing countries',
 'Applied Science',
 'Law',
 'City Planning',
 'Ingeniería de Sistemas',
 'Relaciones Internacionales',
 'Management',
 'Education - Second Lang

In [133]:
col[col['Posgrado'].str.contains('LLM')]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino
111,2001,Guillermo Jimenez Montiel,Masculino,Cundinamarca,JAVERIANA,Bogotá,Abogado,THE UNIVERSITY of SHEFFIELD,Reino Unido,uk,Especialización,"LLM IN INTERNATIONAL, COMMERCIAL AND EUROPEAN LAW","Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,54.702354,-3.276575,"54.7023545,-3.2765753",Allerdale
889,2006,Carolina Ramírez Pérez,Femenino,Cundinamarca,Universidad del Rosario,Bogotá,Abogado,Universiteit Van Amsterdam,Países Bajos,amsterdam,Maestría,LLM in International &amp; European Law,"Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,52.373080,4.892453,"52.3730796,4.8924534",Amsterdam
904,2006,Sandra Paola Quintero Carrillo,Femenino,DC,Universidad de la Sabana,Chia,Abogado,Erasmus University Rotterdam,Países Bajos,rotterdam,Maestría,LLM,"Derecho, Ciencias Políticas y Relaciones Inter...",seleccionado,51.924442,4.477750,"51.9244424,4.47775",Rotterdam
2102,2009,Carlos Andrés Rubio Luna,Masculino,Distrito Capital,Universidad Externado De Colombia,Bogotá,Abogado,University of Illinois,EE.UU,urbana,Maestría,LLM,Derecho,beneficiario,40.111717,-88.207301,"40.1117174,-88.207301",Urbana
3026,2010,Cristina Moure Vieco,Femenino,Distrito Capital,Universidad De Los Andes,Bogotá,Abogado,Boston University,EE.UU,boston,Maestría,LLM in American Law,Derecho,beneficiario,42.355433,-71.060511,"42.3554334,-71.060511",Boston
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19248,2022,Ludovica Pascarella Pisani,Femenino,Lacio,Pontificia Universidad Javeriana (PUJ),Bogotá,Abogada,Tilburg University,Países Bajos,tilburgo,Maestría,LLM : Law and Technology,Derecho,seleccionado,51.585618,5.066062,"51.58561845,5.066061613136768",Tilburg
19256,2022,Juan Ignacio Guerra Toro,Masculino,Bogotá,Universidad de Los Andes,Bogotá,Abogado,Northwestern University,EE.UU,chicago,Maestría,LLM,Derecho,seleccionado,41.875562,-87.624421,"41.8755616,-87.6244212",Chicago
19266,2022,Margie Dayanna Parra Morales,Femenino,Bogotá,Universidad Militar Nueva Granada (UMNG),Bogotá,Abogada,Georgetown University,EE.UU,washington d. c.,Maestría,LLM,Derecho,seleccionado,38.895037,-77.036543,"38.8950368,-77.0365427",Washington
19278,2022,Andrés Felipe Melo Paz,Masculino,Bogotá,Universidad de Los Andes,Bogotá,Abogado,King’s College London,Reino Unido,londres,Maestría,LLM,Derecho,beneficiario,51.507322,-0.127647,"51.5073219,-0.1276474",London


In [134]:
col['Posgrado'] = col['Posgrado'].str.replace('M\.B\.A\.','MBA')

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_16668/1127237989.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Posgrado'] = col['Posgrado'].str.replace('M\.B\.A\.','MBA')


In [135]:
col['Posgrado'] = col['Posgrado'].str.replace('LL\.M\.','LLM')

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_16668/1190741977.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Posgrado'] = col['Posgrado'].str.replace('LL\.M\.','LLM')


In [139]:
col['Posgrado'] = col['Posgrado'].str.replace('LL\.M','LLM')

C:\Users\SRODRI~1\AppData\Local\Temp/ipykernel_16668/1642057101.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  col['Posgrado'] = col['Posgrado'].str.replace('LL\.M','LLM')


In [143]:
import re

In [189]:
def get_ab(line):
    tag  = re.findall('\(.*?\)',line)
    if tag and len(tag) == 1:
        tag = tag[0]
        inside = str(tag).replace('(','')
        inside = str(inside).replace(')','')
        line = str(line).replace(tag, '- '+inside)
    line = re.sub(' +', ' ',line)
    return line

In [191]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].apply(lambda x: get_ab(x))

Carnegie Mellon University
Pratt Institute
Carnegie Mellon University
Carnegie Mellon University
(EHESS)
École des Hautes Études en Sciences Sociales - EHESS
(IE)
Instituto de Empresa - IE
(UCL)
University College London - UCL
London School of Economics Political Science
University of York
Columbia University
Georgia Institute of Technology
McGill University
Johns Hopkins University
Georgetown University
University of North Carolina
Florida Internacional University
Imperial College
University of Leeds
Carleton University
University of Essex
University of Stirling
Parsons School of Desing
University of Sydney
Indiana University
Harvard University
Georgetown University
University of Texas
Carlos III de Madrid
University of Rochester
University of Manchester
Tecnische Univertität Hamburg-Harburg
The New School University
Georgetown University
Georgetown University
University of South Carolina
The New School
Texas Christian University
Massachusetts Institute of Technology - MIT
McGill Univ

Universität Leipzig
Tilburg University
University of Chicago
(LSE)
London School of Economics Political Science - LSE
Politecnico di Milano
University of Cambridge
Universidad de Zaragoza
University of Kent
Harvard University
University of Oxford
University of North Carolina
University of Leeds
University of Washington
University of Leeds
University of Wisconsin
University of Southampton
Tsinghua University
Politecnico di Torino
Columbia University
Columbia University
Politecnico di Milano
Politecnico di Torino
Université de Genève
Pennsylvania State University
(UCL)
University College London - UCL
University of Melbourne
Université de Genève
University of Adelaide
Federazione Italiana Hockey E Pattinaggio
(Esade)
Escuela Superior de Administración de Empresas - Esade
Hong Kong University of Science And Technology
Universidad Politécnica de Valencia
Politecnico di Milano
Sciences Po
University of Cincinnati
Georgetown University
Boston University
University of Melbourne
University of I

University of Greenwich
City University London
(UCL)
University College London - UCL
Université de Montreal
Pratt Institute
Karlsruher Institut für Technologie
London School of Hygiene Tropical Medicine
Technische Universität Berlin
Eidgenössische Technische Hochschule
Delft University of Technology
École des Hautes Études Commerciales - Hec
California Institute of the Arts
UB - Universidad de Barcelona
Université Vincennes Saint Denis - Paris 8
Heriot Watt University
Columbia University
Sciences Po
New York University
Columbia University
Columbia University
(UCL)
University College London - UCL
École National Supérieure d'Architecture de Grenoble
University of Manchester
University of Oxford
UB - Universidad de Barcelona
Università commerciale Luigi Bocconi
IE - Instituto Empresa
Technische Universität München
Lancaster University
University of Sydney
Université Toulouse 1 - Capitole
World Maritime University
Queen Mary
Hertie School of Governance
Emory University
Delft University of 

Universiteit Maastricht
University of Oxford
University of California, Berkeley
University of California, Los Ángeles
London School of Hygiene Tropical Medicine
Politecnico di Torino
Rice University
University of Leicester
Audencia Nantes École de Management
The New School
(NYU)
New York University - NYU
École D'Économie de Paris
Imperial College London
Virginia Polytechnic Institute and State University - Virginia Tech
Hochschule für Musik Saar
Internationale Filmschule Köln
University of Warwick
Université Laval
(Unicamp)
Universidade Estadual de Campinas - Unicamp
(IE)
Instituto de Empresa - IE
Chalmers University of Technology
Queens College
National University of Ireland, Galway
University of Warwick
Erasmus Universiteit Rotterdam
Université de Lausanne
Massachusetts Institute of Technology - MIT
Universiteit Twente
University of Nottingham
Hochschule Anhalt
Harvard University
Technische Universität Hamburg-Harburg
Stony Brook University
Friedrich Alexander Universität Erlangen Nü

Monash University
Virginia Polytechnic Institute and State University
Universiteit Maastricht
The University of Edinburgh
(LMU)
Ludwig-Maximilians-Universität München - LMU
London Business School
(UCL)
University College London - UCL
The University of Edinburgh
University of Manchester
Hochschule für Musik und Theater München
(LSE)
London School of Economics Political Science - LSE
Texas AM University
Institut d'Études Politiques de Paris - Sciences Po
University of British Columbia
Sorbonne Université
Universität Hohenheim
Harvard University
University of Texas at Austin
Queen Mary University of London - QMUL
Albert-Ludwigs-Universität Freiburg
Goldsmiths College, University of London
Georgetown University
(CalArts)
California Institute of the Arts - CalArts
King’s College London
Georg-August-Universität Göttingen
Universiteit Gent
Malmö University
Australian National University
Purdue University
Clemson University
(UMD)
University of Maryland - UMD
University of the Arts London - UAL

University of the Arts London
The University of Auckland
The University of Queensland - UQ
(UB)
Universitat de Barcelona – Universidad de Barcelona - UB
Norwegian University of Science and Technology - NTNU
Harvard University
London School of Economics and Political Science
Universität Stuttgart
New York University
(Heidelberg University)
Ruprecht-Karls-Universität Heidelberg - Heidelberg University
Technical University of Denmark - DTU
Technische Universität Kaiserslautern
The University of Texas at Austin
Queen Mary University of London
(RWTH Rheinisch)
Westfälische Technische Hochschule Aachen - RWTH Rheinisch
Universität Stuttgart
Monash University
Technische Universität München - TUM
Columbia University in the City of New York
Erasmus University Rotterdam
IHE Delft Institute for Water Education
Georg- August- Universität Göttingen
Technische Universität Berlin
The University of British Columbia - UBC
Columbia University in the City of New York
University of Warwick
Hertie School o

University of Notre Dame
University College London
Ghent University
European Institute of Innovation Technology - KIC InnoEnergy Master School
University of Cambridge
(QUT)
Queensland University of Technology - QUT
(UNA)
Universidad de Navarra - UNA
London School of Economics and Political Science
University of Chicago
University of Pennsylvania
Cornell University
(VU)
Vrije Universiteit Amsterdam - VU
PSB Paris Business School
Politecnico di Milano
(QUT)
Queensland University of Technology - QUT
Delft University of Technology
New York University
(UNA)
Universidad de Navarra - UNA
Durham University
Imperial College London
University of Melbourne
Michigan State University – Michigan State University - Detroit College of Law
University of Leeds
(UPM)
Universidad Politécnica de Madrid - UPM
London School of Economics and Political Science
Erasmus University Rotterdam
Georgetown University
London School of Economics and Political Science
(UQ)
The University of Queensland - UQ
Politecnico d

In [195]:
col['Univ. Posgrado'].unique().tolist()

['Carnegie Mellon University',
 'Pratt Institute',
 'École des Hautes Études en Sciences Sociales - EHESS',
 'Instituto de Empresa - IE',
 'University College London - UCL',
 'London School of Economics Political Science',
 'University of York',
 'Columbia University',
 'Georgia Institute of Technology',
 'McGill University',
 'Johns Hopkins University',
 'Georgetown University',
 'University of North Carolina',
 'Florida Internacional University',
 'Imperial College',
 'University of Leeds',
 'Carleton University',
 'University of Essex',
 'University of Stirling',
 'Parsons School of Desing',
 'University of Sydney',
 'Indiana University',
 'Harvard University',
 'University of Texas',
 'Carlos III de Madrid',
 'University of Rochester',
 'University of Manchester',
 'Tecnische Univertität Hamburg-Harburg',
 'The New School University',
 'University of South Carolina',
 'The New School',
 'Texas Christian University',
 'Massachusetts Institute of Technology - MIT',
 'University of To

In [174]:
get_ab('Matt sfddsfa (MIT) y por eso esta (si) funciona')

found  ['(MIT)', '(si)']


In [178]:
str("(hola)").replace('(','')

'hola)'

In [196]:
col['Univ. Posgrado'].unique().tolist()

['Carnegie Mellon University',
 'Pratt Institute',
 'École des Hautes Études en Sciences Sociales - EHESS',
 'Instituto de Empresa - IE',
 'University College London - UCL',
 'London School of Economics Political Science',
 'University of York',
 'Columbia University',
 'Georgia Institute of Technology',
 'McGill University',
 'Johns Hopkins University',
 'Georgetown University',
 'University of North Carolina',
 'Florida Internacional University',
 'Imperial College',
 'University of Leeds',
 'Carleton University',
 'University of Essex',
 'University of Stirling',
 'Parsons School of Desing',
 'University of Sydney',
 'Indiana University',
 'Harvard University',
 'University of Texas',
 'Carlos III de Madrid',
 'University of Rochester',
 'University of Manchester',
 'Tecnische Univertität Hamburg-Harburg',
 'The New School University',
 'University of South Carolina',
 'The New School',
 'Texas Christian University',
 'Massachusetts Institute of Technology - MIT',
 'University of To

In [198]:
col['Univ. Posgrado'].value_counts()

Politecnico di Milano                                                540
University College London - UCL                                      394
Columbia University                                                  387
London School of Economics Political Science - LSE                   333
Harvard University                                                   331
                                                                    ... 
Brandenburgische Technische Universität Cottbus Senftenberg - BTU      1
Université De Liège En Liège                                           1
Claremont Graduate University                                          1
New Jersey Institute of Technology                                     1
The Hong Kong University of Science And Technology - HKUST             1
Name: Univ. Posgrado, Length: 2410, dtype: int64

In [199]:
def upcase(phrase):
    phrase = phrase.lower()
    arr = phrase.split()
    for i in range(len(arr)):
        arr[i] = arr[i].title()
    return " ".join(arr)

In [202]:
col['Univ. Posgrado'] = col['Univ. Posgrado'].apply(lambda x: upcase(x))

In [206]:
col['Univ. Posgrado'].unique().tolist()

['Carnegie Mellon University',
 'Pratt Institute',
 'École Des Hautes Études En Sciences Sociales - Ehess',
 'Instituto De Empresa - Ie',
 'University College London - Ucl',
 'London School Of Economics Political Science',
 'University Of York',
 'Columbia University',
 'Georgia Institute Of Technology',
 'Mcgill University',
 'Johns Hopkins University',
 'Georgetown University',
 'University Of North Carolina',
 'Florida Internacional University',
 'Imperial College',
 'University Of Leeds',
 'Carleton University',
 'University Of Essex',
 'University Of Stirling',
 'Parsons School Of Desing',
 'University Of Sydney',
 'Indiana University',
 'Harvard University',
 'University Of Texas',
 'Carlos Iii De Madrid',
 'University Of Rochester',
 'University Of Manchester',
 'Tecnische Univertität Hamburg-Harburg',
 'The New School University',
 'University Of South Carolina',
 'The New School',
 'Texas Christian University',
 'Massachusetts Institute Of Technology - Mit',
 'University Of To

In [207]:
col.to_csv('ColfSelecClean.csv',index=False)

In [224]:
data_unis = col[['Univ. Posgrado','País']].to_dict(orient='tight')['data']

In [233]:
col['País'].unique()

array(['EE.UU', 'Francia', 'España', 'Reino Unido', 'Canadá', 'Australia',
       'Alemania', 'Países Bajos', 'Suecia', 'México', 'Brasil',
       'Nueva Zelanda', 'Dinamarca', 'Italia', 'Rusia', 'Suiza',
       'Bélgica', 'Cuba', 'Costa Rica', 'Escocia', 'Argentina', 'China',
       'Chile', 'Austria', 'Irlanda', 'Finlandia', 'Islandia', 'Hungría',
       'República de Singapur', 'Mónaco', 'Nicaragua', 'Noruega',
       'Luxemburgo', 'Estonia', 'Gales', 'Portugal', 'Sudáfrica',
       'República Checa', 'Israel', 'Malasia', 'Puerto Rico', 'Japón',
       'Egipto', 'Emiratos Árabes Unidos', 'Polonia',
       'Bosnia y Herzegovina', 'India', 'Perú', 'Ecuador', 'Singapur',
       'Líbano', 'Eslovenia', 'Hong Kong'], dtype=object)

In [234]:
countries = {'EE.UU':'United States', 'Francia':'France', 'España':'Spain', 'Reino Unido':'England', 'Canadá':'Canada', 'Australia':'Australia',
       'Alemania':'Germany', 'Países Bajos':'Netherlands', 'Suecia':'Sweden', 'México':'Mexico', 'Brasil':'Brazil',
       'Nueva Zelanda':'New Zeland', 'Dinamarca':'Denmark', 'Italia':'Italy', 'Rusia':'Russia', 'Suiza':'Switzerland',
       'Bélgica':'Belgium', 'Cuba':'Cuba', 'Costa Rica':'Costa Rica', 'Escocia':'Scotland', 'Argentina':'Argentina', 'China':'China',
       'Chile':'Chile', 'Austria':'Austria', 'Irlanda':'Irland', 'Finlandia':'Finland', 'Islandia':'Island', 'Hungría':'Hungary',
       'República de Singapur':'Singapore', 'Mónaco':'Monaco', 'Nicaragua':'Nicaragua', 'Noruega':'Norway',
       'Luxemburgo':'Luxembourg', 'Estonia':'Estonia', 'Gales':'Wales', 'Portugal':'Portugal', 'Sudáfrica':'South Africa',
       'República Checa':'Czech Republic', 'Israel':'Israel', 'Malasia':'Malaysia', 'Puerto Rico':'Puerto Rico', 'Japón':'Japan',
       'Egipto':'Egypt', 'Emiratos Árabes Unidos':'United Arab Emirates', 'Polonia':'Poland',
       'Bosnia y Herzegovina':'Bosnia and Herzegovina', 'India':'India', 'Perú':'Peru', 'Ecuador':'Ecuador', 'Singapur':'Singapore',
       'Líbano':'Lebanon', 'Eslovenia':'Slovenia', 'Hong Kong':'Hong Kong'}

In [209]:
unique_u = col['Univ. Posgrado'].unique().tolist()

In [235]:
real_names = {}
for i in data_unis:
    ru = uni.lucky(name = i[0],country=countries[i[1]])
    if ru:
        real_names[(i[0],i[1])] = ru.name
    else:
        real_names[(i[0],i[1])] = i[0]

In [236]:
real_names

{('Carnegie Mellon University', 'EE.UU'): 'Carnegie Mellon University',
 ('Pratt Institute', 'EE.UU'): 'Pratt Institute',
 ('École Des Hautes Études En Sciences Sociales - Ehess',
  'Francia'): 'École Des Hautes Études En Sciences Sociales - Ehess',
 ('Instituto De Empresa - Ie', 'España'): 'Instituto De Empresa - Ie',
 ('University College London - Ucl',
  'Reino Unido'): 'University College London - Ucl',
 ('London School Of Economics Political Science',
  'Reino Unido'): 'London School Of Economics Political Science',
 ('University Of York', 'Reino Unido'): 'University Of York',
 ('Columbia University', 'EE.UU'): 'Columbia University',
 ('Georgia Institute Of Technology',
  'EE.UU'): 'Georgia Institute of Technology',
 ('Mcgill University', 'Canadá'): 'McGill University',
 ('Johns Hopkins University', 'EE.UU'): 'Johns Hopkins University',
 ('Georgetown University', 'EE.UU'): 'Georgetown University',
 ('University Of North Carolina',
  'EE.UU'): 'University of North Carolina at Green

In [237]:
def get_new_uni(row):
    return real_names[(row['Univ. Posgrado'],row['País'])]

In [238]:
col['UniPos2'] = col.apply(lambda x: get_new_uni(x),axis=1)

In [241]:
col[['UniPos2','Univ. Posgrado']]

,UniPos2,Univ. Posgrado
0,Carnegie Mellon University,Carnegie Mellon University
1,Pratt Institute,Pratt Institute
2,Carnegie Mellon University,Carnegie Mellon University
3,Carnegie Mellon University,Carnegie Mellon University
4,École Des Hautes Études En Sciences Sociales -...,École Des Hautes Études En Sciences Sociales -...
...,...,...
19301,Universidade Nova De Lisboa - Unl,Universidade Nova De Lisboa - Unl
19302,Goldsmiths - University Of London,Goldsmiths - University Of London
19303,McMaster University,Mcmaster University
19304,National Film And Television School,National Film And Television School


In [244]:
col.loc[col['UniPos2'] != col['Univ. Posgrado']]

,Prom,Nombre,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino,UniPos2
10,2000,Leonardo Augusto Dueñas Osorio,Masculino,Distrito Capital,Universidad De La Salle,Bogotá,Ingeniero Civil,Georgia Institute Of Technology,EE.UU,atlanta,Doctorado,Civil Engineering,Ingeniería,beneficiario,33.748992,-84.390264,"33.7489924,-84.3902644",Atlanta,Georgia Institute of Technology
11,2000,Nicolás Gómez Osorio,Masculino,Distrito Capital,Universidad De Los Andes,Bogotá,Ingeniero Mecánico,Mcgill University,Canadá,montreal,Maestría en Administración,MBA,Administración y Negocios,beneficiario,45.503182,-73.569806,"45.5031824,-73.5698065",Montréal,McGill University
14,2000,Gloria Cecilia Mejía Delgado,Femenino,Antioquia,Universidad De Antioquia,Medellín,Odontóloga,University Of North Carolina,EE.UU,chapel hill,Doctorado,Epidemiology,Ciencias de la Salud,beneficiario,35.913154,-79.055780,"35.9131542,-79.05578",Chapel Hill,University of North Carolina at Greensboro
22,2000,Luis Felipe González Toro,Masculino,Antioquia,Universidad Pontificia Bolivariana,Bogotá,Ingeniero Mecánico,University Of Sydney,Australia,sydney,Maestría,Aeronautical Engineering,Ingeniería,beneficiario,-33.869844,151.208285,"-33.8698439,151.2082848",Sydney,University of Sydney
26,2000,Ana Carolina Pereira Restrepo,Femenino,Distrito Capital,Universidad De La Sabana,Chía,Comunicador Social - Periodista,University Of Texas,EE.UU,austin,Maestría en Administración,MBA,Administración y Negocios,beneficiario,30.271129,-97.743700,"30.2711286,-97.7436995",Austin,University of Texas Medical Branch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19284,2022,Brayan Mauricio Méndez Camacho,Masculino,Bogotá,Universidad Nacional de Colombia (UNAL),Bogotá,Lingüista,University Of Groningen,Países Bajos,groninga,Maestría,Linguistics: Multilingualism,Ciencias Sociales,seleccionado,53.219065,6.568008,"53.2190652,6.5680077",Groningen,University of Groningen
19285,2022,Lina Berón Echavarría,Femenino,Pichincha,Kenyon College,Gambier,Bachelor of Arts,University Of Pennsylvania,EE.UU,filadelfia,Maestría,Landscape Architecture,Arquitectura y Diseño,seleccionado,39.952724,-75.163526,"39.9527237,-75.1635262",Philadelphia,University of Pennsylvania
19293,2022,Santiago Gutiérrez Ruiz,Masculino,Bogotá,Universidad de Los Andes,Bogotá,Ingeniero Civil,University Of Chicago,EE.UU,chicago,Maestría,MBA,Administración y Negocios,beneficiario,41.875562,-87.624421,"41.8755616,-87.6244212",Chicago,University of Chicago
19295,2022,Álvaro José Guijarro May,Masculino,Atlántico,Universidad del Norte (Uninorte),Barranquilla,Ingeniero Industrial,Hertie School Of Governance,Alemania,berlin,Maestría,Data Science for Public Policy,Ingeniería,beneficiario,52.517037,13.388860,"52.5170365,13.3888599",Berlin,Hertie School of Governance


In [247]:
len(col['Univ. Posgrado'].unique())

2269

In [248]:
len(col['UniPos2'].unique())

2261

In [250]:
col.drop(['Nombre'],axis=1,inplace=True)

In [252]:
col.to_csv('ColfSelecClean.csv',index=False)

In [253]:
col

,Prom,Género,Departamento,Univ. Pregrado,Ciudad Pregrado,Pregrado,Univ. Posgrado,País,Ciudad Destino,Tipo,Posgrado,Área,Estado,lat,lon,loc,Ciudad_Destino,UniPos2
0,2000,Masculino,Valle,Pontificia Universidad Javeriana,Bogotá,Ingeniero Industrial,Carnegie Mellon University,EE.UU,pittsburgh,Maestría en Administración,Industrial Administration,Administración y Negocios,beneficiario,40.441694,-79.990086,"40.4416941,-79.9900861",Pittsburgh,Carnegie Mellon University
1,2000,Femenino,Distrito Capital,Universidad De Los Andes,Bogotá,Maestra en Textiles,Pratt Institute,EE.UU,brooklyn,Maestría,Industrial design,Ciencias Básicas,beneficiario,40.652601,-73.949721,"40.6526006,-73.9497211",City of New York,Pratt Institute
2,2000,Femenino,Distrito Capital,Universidad De Los Andes,Bogotá,Ingeniero de Sistemas,Carnegie Mellon University,EE.UU,pittsburgh,Maestría en Administración,MBA Industrial Administration,Administración y Negocios,beneficiario,40.441694,-79.990086,"40.4416941,-79.9900861",Pittsburgh,Carnegie Mellon University
3,2000,Masculino,Bolívar,Universidad De Los Andes,Bogotá,Economista,Carnegie Mellon University,EE.UU,pittsburgh,Maestría en Administración,Information Systems Management,Administración y Negocios,beneficiario,40.441694,-79.990086,"40.4416941,-79.9900861",Pittsburgh,Carnegie Mellon University
4,2000,Masculino,Distrito Capital,Universidad Externado De Colombia,Bogotá,Finanzas Y Relaciones Internacionales,École Des Hautes Études En Sciences Sociales -...,Francia,paris,Doctorado,Sociología,Ciencias Sociales,beneficiario,48.858890,2.320041,"48.8588897,2.3200410217200766",Paris,École Des Hautes Études En Sciences Sociales -...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19301,2022,Masculino,Santander,"Universidad de San Buenaventura, seccional Cali",Cali,Ingeniero Agroindustrial,Universidade Nova De Lisboa - Unl,Portugal,lisboa,Maestría,Management,Administración y Negocios,seleccionado,38.707751,-9.136592,"38.7077507,-9.1365919",Lisboa,Universidade Nova De Lisboa - Unl
19302,2022,Femenino,Bogotá,Universidad de Los Andes,Bogotá,Maestra en Artes,Goldsmiths - University Of London,Reino Unido,londres,Maestría,Children’s Literature: Children’s Book Illustr...,Artes,beneficiario,51.507322,-0.127647,"51.5073219,-0.1276474",London,Goldsmiths - University Of London
19303,2022,Masculino,Bogotá,Fundación Universitaria Juan N. Corpas,Bogotá,Médico y Cirujano General,Mcmaster University,Canadá,hamilton,Especialización,Advanced Coronary Imaging,Ciencias de la Salud,beneficiario,43.256080,-79.872858,"43.2560802,-79.8728583",Hamilton,McMaster University
19304,2022,Masculino,Bogotá,Universidad Externado de Colombia,Bogotá,Comunicador Social y Periodista,National Film And Television School,Reino Unido,beaconsfield,Maestría,Directing Documentary,Artes,beneficiario,51.608288,-0.657123,"51.6082881,-0.6571233",Beaconsfield,National Film And Television School


In [62]:
import spacy
nlp = spacy.load("en_core_web_md")

In [68]:
doc = nlp('Columbia U.')

In [69]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Columbia 0 8 ORG
